In [1]:

from pymongo import MongoClient
import os
import base64  # Import base64 module
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd

# Define the AnimalShelterCRUD class for database interaction
class AnimalShelterCRUD:
    def __init__(self, username=None, password=None):
        mongo_host = os.environ.get('MONGO_HOST', 'nv-desktop-services.apporto.com')
        mongo_port = int(os.environ.get('MONGO_PORT', 30055))

        self.client = MongoClient(host=mongo_host, port=mongo_port, username=username, password=password, authSource='admin')
        self.db = self.client['AAC']
        self.collection = self.db['animals']

    def query_animals(self, query):
        try:
            cursor = self.collection.find(query)
            result_list = [animal for animal in cursor]
            return result_list
        except Exception as e:
            print(f"Error querying animal documents: {e}")
            return []

# Initialize the Dash app
app = JupyterDash(__name__)

# Initialize the AnimalShelter object
username = "aacuser"
password = "MySnhu"
db = AnimalShelterCRUD(username, password)

# Read data from MongoDB into a pandas DataFrame
df = pd.DataFrame.from_records(db.query_animals({}))

# Drop the '_id' column
df.drop(columns=['_id'], inplace=True)

# Specify the path to the image file on your desktop
image_path = '/home/coreyhamilton_snhu/Desktop/Grazioso Salvare Logo.png'

# Read the image file and encode it as base64
with open(image_path, 'rb') as f:
    encoded_image = base64.b64encode(f.read()).decode('utf-8')

# Set up the Dash app layout with CSS styling for colors
app.layout = html.Div([
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'width': '200px', 'height': '200px'})),
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard - Corey Hamilton'))),
    html.Hr(),
    # Interactive filtering options
    dcc.Dropdown(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
        ],
        value=None,
        placeholder="Select a filter...",
        style={'color': 'black'}  # Set dropdown text color to black
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        selected_rows=[0],  # Select the first row by default
        row_selectable='single',  # Enable single-row selection
        # FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        style_cell={'backgroundColor': 'rgb(240, 240, 240)', 'color': 'black'}  # Set table cell background to light gray and text color to black
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        id='map-id',
        className='col s12 m6',
    ),
    html.Div(
        id='graph-id'  # Add the div for the graph with the specified ID
    )
], style={'backgroundColor': 'rgb(240, 240, 240)', 'color': 'black'})  # Set overall background to light gray and text color to black

# Callback to update the data table based on filter selection
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type is None:
        return df.to_dict('records')
    else:
        # Filter the DataFrame based on the selected filter value and age criteria
        filtered_df = df[(df['breed'].isin(get_breeds_for_rescue_type(filter_type))) & (pd.to_numeric(df['age_upon_outcome'].str.split(' ', expand=True)[0]) < 3)]
        return filtered_df.to_dict('records')




# Callback Routine for Geolocation Chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    # Add code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)

    # Check if the DataFrame is empty
    if dff.empty:
        return []

    # Because we only allow single-row selection, the list can be converted to a row index here
    if index is None or not index:
        row = 0
    else:
        row = index[0]

    # Check if the row index is within bounds
    if row < 0 or row >= len(dff):
        return []

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   # Marker with tool tip and popup
                   # Column 13 and 14 define the grid-coordinates for the map
                   # Column 4 defines the breed for the animal
                   # Column 9 defines the name of the animal
                   dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                             children=[
                                 dl.Tooltip(dff.iloc[row, 4]),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(dff.iloc[row, 9])
                                 ])
                             ])
               ])
    ]

# Callback to update the second chart based on the selected data entry
@app.callback(
    Output('graph-id', "children"),
    [Input('filter-type', 'value')])
def update_chart(filter_type):
    if filter_type is None:
        return []

    filtered_df = df[df['breed'].isin(get_breeds_for_rescue_type(filter_type))]
    
    # Example: Create a pie chart based on the filtered data
    return dcc.Graph(
        figure={
            'data': [
                {'labels': filtered_df['breed'], 'values': [1]*len(filtered_df), 'type': 'pie', 'name': 'Breed Distribution'},
            ],
            'layout': {
                'title': f'Breed Distribution for {filter_type}'
            }
        }
    )

def get_breeds_for_rescue_type(rescue_type):
    if rescue_type == 'Water Rescue':
        return ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
    elif rescue_type == 'Mountain or Wilderness Rescue':
        return ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']
    elif rescue_type == 'Disaster or Individual Tracking':
        return ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
    else:
        return []

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)



























Dash is running on http://127.0.0.1:10511/

Dash app running on http://127.0.0.1:10511/
